In [0]:
spark.conf.set(
  "fs.azure.account.key.eightfiveadls.dfs.core.windows.net",
  "OCLTgHfNR0IxBpUNlohvKbdO6FxTc5EJoOO/UuOaQrU1S4p6vt9FLbWVSRXswGHcllZpt3hO8s4f+ASt0jJlJw=="
)

In [0]:
iowa_df_create = spark.read.format("parquet").load('abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_parquet/')
iowa_df_append = spark.read.format("parquet").load('abfss://<container>@<dlsname>.dfs.core.windows.net/IowaLiquorSalesdf2022.parquet')

In [0]:
iowa_df_create.write \
    .format('delta') \
    .mode('overwrite') \
    .partitionBy('Date') \
    .option('mergeSchema', 'false') \
    .save('abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/')

In [0]:
iowa_df_append.write \
    .format('delta') \
    .mode('append') \
    .saveAsTable('iowa.iowa_delta_table')

In [0]:
from delta.tables import *
DeltaTable.createIfNotExists(spark).tableName("iowa.iowa_delta_table").location('abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/').execute()

In [0]:
%sql
spark.sql("CREATE TABLE iowa.iowa_delta_table USING DELTA LOCATION 'abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/'")

In [0]:
display(spark.sql("DESCRIBE TABLE EXTENDED iowa.iowa_delta_table"))

In [0]:
DeltaTable.isDeltaTable(spark, 'abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/')

In [0]:
#From a table path
delta_table_01 = DeltaTable.forPath(spark, 'abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/')
#From a table name
delta_table_02 = DeltaTable.forName(spark, 'iowa.iowa_delta_table')

In [0]:
delta_df_01 = delta_table_02.toDF()
delta_df_02 = spark.read.format('delta').load('abfss://<container>@<dlsname>.dfs.core.windows.net/iowa_delta_table/')
delta_df_03 = spark.read.table('iowa.iowa_delta_table')
delta_df_04 = spark.table('iowa.iowa_delta_table')

In [0]:
display(delta_df_01.take(10))

In [0]:
spark.sql("CACHE SELECT idt.Date, idt.City, idt.SaleDollars FROM iowa.iowa_delta_table AS idt WHERE YEAR(idt.Date) = 2022 AND MONTH(idt.Date) = 2")

In [0]:
spark.sql("ALTER TABLE iowa.iowa_delta_table SET TBLPROPERTIES (delta.autoOptimize.optimizeWrite = true, delta.autoOptimize.autoCompact = true)")

In [0]:
delta_table_01.optimize().executeCompaction()
delta_table_01.optimize().where("Date between '2022-01-01' and '2022-12-31'")
spark.sql("OPTIMIZE iowa.iowa_delta_table ZORDER BY City")

In [0]:
%sql
OPTIMIZE iowa.iowa_delta_table
OPTIMIZE iowa.iowa_delta_table WHERE Date between '2022-01-01' and '2022-12-31'
OPTIMIZE iowa.iowa_delta_table ZORDER BY City

In [0]:
delta_table_01.upgradeTableProtocol(1,2)

In [0]:
%sql
ALTER TABLE iowa.iowa_delta_table SET TBLPROPERTIES('delta.minReaderVersion' = '1', 'delta.minWriterVersion' = '3')

In [0]:
delta_table_01.vacuum(168) #7 days

In [0]:
%sql
VACUUM iowa.iowa_delta_table RETAIN 168 HOURS

In [0]:
display(delta_table_01.history())

In [0]:
%sql
DESCRIBE HISTORY iowa.iowa_delta_table

In [0]:
df_with_version = spark \
    .read.format('delta') \
    .option('versionAsOf', 2) \
    .table('iowa.iowa_delta_table')
    
#.option('timestampAsOf', '2022-08-12T06:55:52.000+0000')

In [0]:
%sql
SELECT * FROM iowa.iowa_delta_table VERSION AS OF 2
SELECT * FROM iowa.iowa_delta_table@v2
SELECT * FROM iowa.iowa_delta_table TIMESTAMP AS OF "2022-08-12T06:55:52.000+0000"

In [0]:
delta_table_01.restoreToVersion(5)
delta_table_01.restoreToTimestamp('2022-08-12T06:55:52.000+0000')

In [0]:
%sql
RESTORE tableName VERSION AS OF 5
RESTORE tableName TIMESTAMP AS OF "2022-08-12T06:55:52.000+0000"

In [0]:
import pyspark.sql.functions as F 

delta_table_01.update("StoreNumber = 3417 and Date = '2022-01-03' and ItemNumber = 64904", {"SaleDollars": '71'})
delta_table_01.update(condition = (F.col("StoreNumber") == 3417) & (F.col("Date") == '2022-01-03') & (F.col("ItemNumber") == 64904), set = {"SaleDollars": '81'})

In [0]:
%sql
UPDATE iowa.iowa_delta_table set SaleDollars = 81 WHERE StoreNumber = 3417 and Date = '2022-01-03' and ItemNumber = 64904

In [0]:
delta_table_01.delete("StoreNumber = 3417 and Date = '2022-01-03' and ItemNumber = 64904")
delta_table_01.delete(condition = (F.col("StoreNumber") == 3417) & (F.col("Date") == '2022-01-03') & (F.col("ItemNumber") == 64904), set = {"SaleDollars": '81'})

In [0]:
%sql
DELETE iowa.iowa_delta_table WHERE StoreNumber = 3417 and Date = '2022-01-03' and ItemNumber = 64904

In [0]:
delta_table_01.alias('dt_org') \
  .merge(
    delta_df_02_to_merge.alias('dt_to_merge'),
    'dt_org.InvoiceItemNumber = dt_to_merge.InvoiceItemNumber and dt_org.SaleDollars <> dt_to_merge.SaleDollars'
  ) \
  .whenMatchedUpdate(set =
    {
      "SaleDollars": "dt_to_merge.SaleDollars",
      "Pack": "dt_to_merge.Pack"
    }
  ) \
  .whenNotMatchedInsertAll() \
  .execute()

In [0]:
%sql
MERGE INTO iowa.iowa_delta_table as dt_target
USING iowa.iowa_delta_table_deep_copy as dt_source
ON dt_target.InvoiceItemNumber = dt_source.InvoiceItemNumber and dt_target.SaleDollars <> dt_source.SaleDollars
WHEN MATCHED THEN
  UPDATE SET
    SaleDollars = dt_source.SaleDollars,
    Pack = dt_source.Pack
WHEN NOT MATCHED
  THEN INSERT (
    InvoiceItemNumber,
    Date,
    StoreNumber,
    StoreName,
    Address,
    City
  --  ....and rest columns
  )
  VALUES (
    dt_source.InvoiceItemNumber,
    dt_source.Date,
    dt_source.StoreNumber,
    dt_source.StoreName,
    dt_source.Address,
    dt_source.City
 --   ...and rest columns
  )